In [5]:
import os
import re
import pandas as pd
from tqdm.notebook import tqdm
import subprocess
from PIL import Image
import numpy as np

In [8]:
df = pd.read_csv('../dataset/SecretPart.csv', index_col=None)
# df.head()

In [9]:
def rename_file(num):
    def fun(row):
        row[0] = f'dataset/sample_{num}/{row[0]}'
        row[0] = row[0].replace('.png', f'_s{num}.png')
        return row
    return fun

def rename_expert_file(file_name):
    file_name = 'dataset/Expert/' + file_name[:-4] + '_expert.png'
    return file_name

In [10]:
df_s1 = df.iloc[:,[0,1]]
df_s1 = df_s1.rename(columns={"Case": "pred_mask_path", "Sample 1": "review"})
df_s1['pred_mask_path'].apply(rename_expert_file)
df_s1['true_mask_path'] = df_s1['pred_mask_path'].apply(rename_expert_file)
df_s1 = df_s1.apply(rename_file(1), axis=1)
df_s1 = df_s1[['true_mask_path', 'pred_mask_path', 'review']]

df_s2 = df.iloc[:,[0,1]]
df_s2 = df_s2.rename(columns={"Case": "pred_mask_path", "Sample 1": "review"})
df_s2['true_mask_path'] = df_s2['pred_mask_path'].apply(rename_expert_file)
df_s2 = df_s2.apply(rename_file(2), axis=1)
df_s2 = df_s2[['true_mask_path', 'pred_mask_path', 'review']]

df_s3 = df.iloc[:,[0,1]]
df_s3 = df_s3.rename(columns={"Case": "pred_mask_path", "Sample 1": "review"})
df_s3['true_mask_path'] = df_s3['pred_mask_path'].apply(rename_expert_file)
df_s3 = df_s3.apply(rename_file(3), axis=1)
df_s3 = df_s3[['true_mask_path', 'pred_mask_path', 'review']]

In [12]:
new_df = pd.concat([df_s1, df_s2, df_s3]).reset_index(drop=True)

In [13]:
new_df

,true_mask_path,pred_mask_path,review
0,dataset/Expert/00011827_003_expert.png,dataset/sample_1/00011827_003_s1.png,3
1,dataset/Expert/00011925_072_expert.png,dataset/sample_1/00011925_072_s1.png,5
2,dataset/Expert/00012045_019_expert.png,dataset/sample_1/00012045_019_s1.png,1
3,dataset/Expert/00012094_040_expert.png,dataset/sample_1/00012094_040_s1.png,5
4,dataset/Expert/00012174_000_expert.png,dataset/sample_1/00012174_000_s1.png,4
...,...,...,...
115,dataset/Expert/00019124_090_expert.png,dataset/sample_3/00019124_090_s3.png,5
116,dataset/Expert/00019495_004_expert.png,dataset/sample_3/00019495_004_s3.png,2
117,dataset/Expert/00019767_016_expert.png,dataset/sample_3/00019767_016_s3.png,4
118,dataset/Expert/00020000_000_expert.png,dataset/sample_3/00020000_000_s3.png,2


In [15]:
import sys
sys.path.insert(0, '..')

In [16]:
from scripts.catboost_predict import get_models_by_names, predict

In [38]:
models = get_models_by_names("cb_mae-0.5818.cbm")

In [39]:
y_pred = []
for i, (true_mask_path, pred_mask_path, target) in new_df.iterrows():
    pred = predict(models, '../' + true_mask_path, '../' + pred_mask_path)
    y_pred.append(pred)

In [40]:
from catboost.utils import eval_metric

In [41]:
eval_metric(new_df['review'].to_numpy(), y_pred, 'MAE')

[1.7499991250000018]